# Flomaster Example: Two Pump

In [ ]:
from IPython.display import display
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import Bounds, differential_evolution, LinearConstraint

from fmu_optimise import create_cost_function

In [ ]:
filename = "Two Pump Model Win.fmu"

# set input parameters with bounds
inputs = {
    "[Major Pipe Diameter]": [0.1, 0.3],
    "[Pipe Diameter]": [0.1, 0.3],
    "[RatedFlow]": [0.05, 0.07],
    "[RatedHead]": [10, 30],
}

# create linear constraint 
con = LinearConstraint([1, -1, 0, 0], 0, np.inf) 

# outputs to be recorded
output = [
    "Primary Pump Power",
    "Secondary Pump Power"
]

# create bounds in expected format
bounds = Bounds([b[0] for b in inputs.values()], [b[1] for b in inputs.values()])

# create post simulation cost function
def post_simulation_function(results):
    return sum([np.trapz(results["Primary Pump Power"]), np.trapz(results["Secondary Pump Power"])])

In [ ]:
# cost function
func = create_cost_function(
    filename=filename,
    input_names=inputs.keys(),
    output_names=output,
    start_time=0,
    stop_time=60,
    step_size=0.5,
    post_simulation_function=post_simulation_function
)

In [ ]:
def unbound_generation_callback(fig, convergence_list, input_names, xk, convergence=0):
    print('Best of generation:')
    [print(f"{k} - {v}") for k, v in zip(input_names, xk)]
    convergence_list.append(convergence)
    fig.data[0].y = convergence_list

fig = go.FigureWidget()
fig.add_scatter()
fig.update_layout(
    title="Convergence Plot",
    xaxis_title="Generations",
    yaxis_title="Convergence"
)
display(fig)
convergence_list = []
generation_callback = lambda xk, convergence=0 : unbound_generation_callback(fig, convergence_list, inputs.keys(), xk, convergence)
# solve
result = differential_evolution(func, bounds, popsize=3, maxiter=20, callback=generation_callback, constraints=(con,))
print("Final output")
print(result)